In [1]:
import numpy as np
import numpy.random as npr
from IPython.display import Latex

In [2]:
npr.seed(10)

sites = np.ones(12)*1/2*np.sign(npr.random(12)-0.5)
links = np.ones(12)*1/2*np.sign(npr.random(12)-0.5)
print(sites)
print(links)

[ 0.5 -0.5  0.5  0.5 -0.5 -0.5 -0.5  0.5 -0.5 -0.5  0.5  0.5]
[-0.5  0.5  0.5  0.5  0.5 -0.5  0.5  0.5  0.5 -0.5 -0.5  0.5]


In [3]:
leg_sites = {
    1/2 : '$\circ$',
    -1/2 : '$\\bullet$'
}

leg_links = {
    1/2 : '$\leftarrow$',
    -1/2 : '$\\rightarrow$'
}

def Visualize(sites, links):
    sites_symbols = np.vectorize(leg_sites.get)(sites)
    links_symbols = np.vectorize(leg_links.get)(links)
    seq_list = [i + j for i, j in zip(sites_symbols, links_symbols)]
    total_list = ''.join(seq_list)
    return total_list

def Gaggiublock(i, sites, links):
    s1, l1 = sites[i],links[i]
    s2, l2 = sites[i+1],links[i+1]
    g = s2 - l2 + l1 + (-1)**(i)*1/2 == 0     
    '''
    Since numering start from 0 instead of 1 like in the review, 
    we absorbe a '-1' sign (exponent of last term becomes i instead of i+1)
    '''
    return g

In [4]:
sigma1 = (-1/2,-1/2)
sigma2 = (1/2,1/2)

sites = sigma1[0], sigma2[0]
links = sigma1[1], sigma2[1]
    
print(sites, links)
    
Gaggiublock(0,sites,links)

(-0.5, 0.5) (-0.5, 0.5)


True

In [5]:
from itertools import permutations, product

all_sit = list(product([0.5,-0.5], repeat=12))
all_lin = list(product([0.5,-0.5], repeat=12))
ok = []

for s in all_sit:
    for l in all_lin:
        check = True
        for i in range(len(s)-1):
            if not Gaggiublock(i, s, l): 
                check = False
                break
        if check == True: ok.append(np.array([s,l]))

In [6]:
n = 10
x = Visualize(ok[n][0],ok[n][1]) 
Latex(x)

<IPython.core.display.Latex object>

In [7]:
len(ok)

754

In [8]:
sequences_flattened=[]
for i in range(len(ok)):
    sequences_flattened.append(ok[i].flatten())

# Definizione della RBM


In [12]:
def activate(v_in, wei, bias, DE, info=False):
    act = np.dot(v_in, wei) + bias
    prob = 1. / (1. + np.exp(-DE*act))
    n = len(act)
    v_out = np.full(n, vmin)
    v_out[np.random.random_sample(n) < prob] = 1
    if info:
        print('input=', v_in)
        print('act=', act)
        print('prob=', prob)
        print('out=', v_out)
        
    return(v_out)

def training_RBM(alpha, w, a, b, seq):
    
    number_sequences = len(seq)
    sequence_length = len(seq[0])
    hidden_units = alpha*sequence_length #setting nubmer of hidden units
    
    #minibatches
    mini, m = int(0.08*number_sequences), 0 
    #epochs
    nepoch = 50
    #l_rate
    l_rate = 1.

    for epoch in range(1, 1+nepoch):
        for n in range(number_sequences):
            if m==0:
                #initialize
                v_data, v_model = np.zeros(sequence_length), np.zeros(sequence_length) 
                h_data, h_model = np.zeros(hidden_units), np.zeros(hidden_units) 
                vh_data, vh_model = np.zeros((sequence_length,hidden_units)), np.zeros((sequence_length, hidden_units)) 

            #CAPIRE BENE IL GAP
            #positive CD phase
            h = activate(v[n],w,b,GAP)
            #negative CD phase
            vf = activate(h, w.T,a,GAP)
            #positive CD nr 2
            hf = activate(vf,w,b, GAP)

            v_data += v[n]
            v_model += vf
            h_data += h
            h_model += hf
            
            theta = np.dot(w.T,seq.T)+b
            
            O_a += seq.T
            O_b += np.tanh(theta)
            O_w += 
            vh_model += np.outer(vf.T, hf)

            m += 1
            if m==mini:
                C = l_rate / mini
                
                dw = C*(vh_data - vh_model)
                da = C*(v_data - v_model)
                db = C*(h_data - h_model)

                w += dw
                a += da
                b += db
                m = 0

        #randomize order
        np.random.shuffle(v)
        l_rate = l_rate / (0.05 * l_rate + 1)
        
    return w,a,b

In [ ]:
#funzioen elementi matrice per Ising e periodic boundaries conditions -> personalizzata

def flip(spin):
    return -spin

def mel_Ising1D_pb(state_S, a, theta):
        flip_seq = [[]] + [[i] for i in range(len(state))]
        output = np.zeros(len(state_s))
        for i in range(len(state_s)-1):
            v = np.zero((len(state_S),1))
            output[i]-=state_s[i]*state_s[i+1]
        
    
    return 
    

In [ ]:
def training_RBM(alpha, w, a, b, seq):
    
    N_s = len(seq) #N -> per avere lo storico, sequenze fatte sin'ora s
    n_spins = len(seq[0]) #L 
    n_h = alpha * n_spins #M
    
    theta = np.dot(w.T, seq.T) + b
    
    Oa = seq.T
    Ob = np.tanh(theta)
    Ow = seq.T.reshape((n_spins, 1, N_s))*np.tanh(theta.reshape(1, n_h, N_s))
    
    #stochastic reconfiguration
    
    
    
    

In [ ]:
        derivs = np.concatenate([dA, dB, dW.reshape(n_spins * n_h, N_s)])

        avg_derivs = np.sum(derivs, axis=1, keepdims=True) / N_s
        avg_derivs_mat = np.conjugate(avg_derivs.reshape(derivs.shape[0], 1))
        avg_derivs_mat = avg_derivs_mat * avg_derivs.reshape(1, derivs.shape[0])
        moment2 = np.einsum('ik,jk->ij', np.conjugate(derivs), derivs) / N_s
        S_kk = np.subtract(moment2, avg_derivs_mat)

        F_p = np.sum(Eloc.transpose() * np.conjugate(derivs), axis=1) / N_s
        F_p -= np.sum(Eloc.transpose(), axis=1) * \
               np.sum(np.conjugate(derivs), axis=1) / (N_s ** 2)
        S_kk2 = np.zeros(S_kk.shape, dtype=complex)
        row, col = np.diag_indices(S_kk.shape[0])
        S_kk2[row, col] = self.lambd(p) * np.diagonal(S_kk)
        S_reg = S_kk + S_kk2
        update = np.dot(np.linalg.inv(S_reg), F_p).reshape(derivs.shape[0], 1)

        return update, derivs

$\theta$ M*N ottenuta da una M*L e una L*N quindi quando si somma B vettore di lunghezza M questo subirà un broadcasting lungo le righe
B: vettore M*1
W: L*M
seq: storico delle sequenze generate: N*L, N cambia a ogni iterazione
corrispondenza con codice kafischer:
n_h -> M
N_s -> N
n_spins -> L

In [138]:
#seq dovrà essere un'array di elementi di questo tipo
seq = np.ones(12)*1/2*np.sign(npr.random(12)-0.5)
alpha = 4
N_s = len(seq)
n_spins = len(seq[0])
n_h = alpha * n_spins

#stdev of initialized weights
sigma = np.sqrt(4. / float(N_s+n_h))

#initialized weights
w = sigma * (2*np.random.rand(n_spins, n_h) - 1) #from -sigma to sigma 
a = sigma * (2*np.random.rand(n_spins) - 1)
b = np.zeros(n_h)



NameError: name 'npr' is not defined

Tommaso ricomincia da qui:


Librerie

In [2]:
import numpy as np
import random
import tqdm

Funzioni

In [277]:
#inizializzo la RBM

def InitializeRBM(n_visible, n_hidden, init_weight):
    '''
    inizializzo completamente la RBM
    '''
    N=n_visible
    M=n_hidden
    #gli a sono i visible bias e sono N
    a = (init_weight * (np.random.rand(N, 1)-0.5)).astype(dtype=complex)
    #i b sono gli hidden bias e sono M
    b = (init_weight * (np.random.rand(M, 1)-0.5)).astype(dtype=complex)
    #i w sono una matrice NxM
    w = (init_weight * (np.random.rand(N, M)-0.5)).astype(dtype=complex)
    return a,b,w


def Demagnetize(current_state, verbose=False):
    '''
    dati una sequenza con magnetizzazione non nulla la porto a magnetizzazione nulla flippando spin in modo intelligente
    '''
    total_mag_init = sum(current_state)
    total_mag=total_mag_init
    
    if total_mag > 0:
        while total_mag != 0:
            current_state[int(np.random.choice(np.where(current_state == 1)[0], 1))] *= -1
            total_mag = sum(current_state)
    elif total_mag < 0:
        while total_mag != 0:
            current_state[int(np.random.choice(np.where(current_state == -1)[0], 1))] *= -1
            total_mag = sum(current_state)
            
    if verbose:
        print('Initial magnetization of the state:', total_mag_init)
        print('Final magnetization of the state:', total_mag)
        
    return(current_state)

def GenerateSpinSequence(n_visible, encoding):
    return np.random.choice(encoding, n_visible, p=[0.5, 0.5]).reshape(n_visible, 1)
    
def SpinSequenceFlips(current_state, n_flips):
    """
    ritorna lo stato iniziale con n_flips spin flippati
    """
    output=np.copy(current_state)
    print(n_flips)
    for i in range(n_flips):
        index = np.random.randint(1,len(current_state))
        output[index]*=-1
    return output
        
def AmplitudeRatio(a, b, w, current_state, proposed_state):
    """ 
    calcola il rapporto tra Psi(state') e Psi(state) dove state' è state con spin flippati
    prende in input i parametri della rbm, lo stato corrente e lo stato proposto dal sampling
    
    Tira fuori la formula C6 
    
    Occhio all'update dei theta in log_proposed_wf perché si updatano con formula C7. Potrebbe esserci qualcosa che non va 
    perché non ho perfettamente capito cosa sia quel 2 W_kj sigma^z_s. Controllate.
    """
    #guardando la documentazione mi ritrovo con la notazione
    thetas=Theta(b,w, current_state)
    log_actual_wf= -np.dot(a.T,current_state)+np.sum(np.log(2*np.cosh(thetas)))
    log_proposed_wf= -np.dot(a.T,current_state)+np.sum(np.log(2*np.cosh(thetas-2*np.dot(w.T,proposed_state))))
    return np.exp(log_proposed_wf/log_actual_wf)

def Theta(b, w, current_state):
    """
    funzione che calcola i theta
    
    """
    return b+np.dot(w.T, current_state)
    
def Metropolis(n_sweeps): #e altro):
    '''
    Generates samples of configurations from an neural quantum state (NQS)
    using a Metropolis-Hastings algorithm.
    '''
    print('Starting Monte Carlo Sampling')

    for i in range(int(n_sweeps)):
        for _ in range(int(sweep_factor * self.n_visible)):
            self.move(n_flips)
        self.current_Hloc = self.local_energy()
        self.state_history.append(np.array(self.current_state))
        self.local_energies.append(self.current_Hloc)
        if self.samples_file:
            self.write_current_state(f)

        print('Completed Monte Carlo Sampling')
    
def optimizer(a, b, w):
    '''
    dati i pesi di una vecchia RBM sputo fuori quelli nuovi
    '''
    pass 

In [278]:
#per forza intero e pari
n_visible=40 
n_hidden=40
#anche Baiesi lo faceva
init_weight=0.001
#da capire bene perché
zero_magnetization = True 
#encoding come spin e non come bit
particle_encoding = [-1,1] 
               
#inizializzo uno stato
current_state = GenerateSpinSequence(n_visible, encoding)

#inizializzo la RBM
a, b, w = InitializeRBM(n_visible, n_hidden, init_weight)

if zero_magnetization == True:
    current_state=Demagnetize(current_state, verbose=True)
               
sampler_params = {'n_sweeps': 10000, 'therm_factor': 0.,'sweep_factor': 1, 'n_flips': 1}

Initial magnetization of the state: [2]
Final magnetization of the state: [0]


In [289]:
#for i in range(100):
#               #ciclo di ottimizzazione
#    #mi aspetto che current state, dato lo stato della RBM e la hamiltoniana propria del modello, venga samplato con metropolis
#    current_state = Metropolis(hamiltonianaIsing1D, RBM_init, zero_magnetization=True, current_state)
#    #a,b,w = optimizer(a,b,w)

In [302]:
current_state = GenerateSpinSequence(n_visible, particle_encoding)
proposed_state= SpinSequenceFlips(current_state, 3)
print((current_state-proposed_state).reshape(n_visible,))
print(AmplitudeRatio(a,b,w,current_state,proposed_state))

3
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 -2 -2  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0]
[[2.7182853+0.j]]
